In [1]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [2]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/loan_top1.csv")

In [3]:
df.printSchema()

root
-- loan_amnt: integer (nullable = true)
-- funded_amnt: integer (nullable = true)
-- funded_amnt_inv: double (nullable = true)
-- term: string (nullable = true)
-- int_rate: double (nullable = true)
-- installment: double (nullable = true)
-- grade: string (nullable = true)
-- sub_grade: string (nullable = true)
-- emp_title: string (nullable = true)
-- emp_length: string (nullable = true)
-- home_ownership: string (nullable = true)
-- annual_inc: double (nullable = true)
-- verification_status: string (nullable = true)
-- issue_d: string (nullable = true)
-- loan_status: string (nullable = true)
-- pymnt_plan: string (nullable = true)
-- purpose: string (nullable = true)
-- title: string (nullable = true)
-- zip_code: string (nullable = true)
-- addr_state: string (nullable = true)
-- dti: double (nullable = true)
-- delinq_2yrs: integer (nullable = true)
-- earliest_cr_line: string (nullable = true)
-- inq_last_6mths: integer (nullable = true)
-- mths_since_last_delinq: integer (nullable = true)
-- mths_since_last_record: integer (nullable = true)
-- open_acc: integer (nullable = true)
-- pub_rec: integer (nullable = true)
-- revol_bal: integer (nullable = true)
-- revol_util: double (nullable = true)
-- total_acc: integer (nullable = true)
-- initial_list_status: string (nullable = true)
-- out_prncp: double (nullable = true)
-- out_prncp_inv: double (nullable = true)
-- total_pymnt: double (nullable = true)
-- total_pymnt_inv: double (nullable = true)
-- total_rec_prncp: double (nullable = true)
-- total_rec_int: double (nullable = true)
-- total_rec_late_fee: double (nullable = true)
-- recoveries: double (nullable = true)
-- collection_recovery_fee: double (nullable = true)
-- last_pymnt_d: string (nullable = true)
-- last_pymnt_amnt: double (nullable = true)
-- next_pymnt_d: string (nullable = true)
-- last_credit_pull_d: string (nullable = true)
-- collections_12_mths_ex_med: integer (nullable = true)
-- mths_since_last_major_derog: integer (nullable = true)
-- policy_code: integer (nullable = true)
-- application_type: string (nullable = true)
-- annual_inc_joint: double (nullable = true)
-- dti_joint: double (nullable = true)
-- verification_status_joint: string (nullable = true)
-- acc_now_delinq: integer (nullable = true)
-- tot_coll_amt: integer (nullable = true)
-- tot_cur_bal: integer (nullable = true)
-- open_acc_6m: integer (nullable = true)
-- open_act_il: integer (nullable = true)
-- open_il_12m: integer (nullable = true)
-- open_il_24m: integer (nullable = true)
-- mths_since_rcnt_il: integer (nullable = true)
-- total_bal_il: integer (nullable = true)
-- il_util: integer (nullable = true)
-- open_rv_12m: integer (nullable = true)
-- open_rv_24m: integer (nullable = true)
-- max_bal_bc: integer (nullable = true)
-- all_util: integer (nullable = true)
-- total_rev_hi_lim: integer (nullable = true)
-- inq_fi: integer (nullable = true)
-- total_cu_tl: integer (nullable = true)
-- inq_last_12m: integer (nullable = true)
-- acc_open_past_24mths: integer (nullable = true)
-- avg_cur_bal: integer (nullable = true)
-- bc_open_to_buy: integer (nullable = true)
-- bc_util: double (nullable = true)
-- chargeoff_within_12_mths: integer (nullable = true)
-- delinq_amnt: integer (nullable = true)
-- mo_sin_old_il_acct: integer (nullable = true)
-- mo_sin_old_rev_tl_op: integer (nullable = true)
-- mo_sin_rcnt_rev_tl_op: integer (nullable = true)
-- mo_sin_rcnt_tl: integer (nullable = true)
-- mort_acc: integer (nullable = true)
-- mths_since_recent_bc: integer (nullable = true)
-- mths_since_recent_bc_dlq: integer (nullable = true)
-- mths_since_recent_inq: integer (nullable = true)
-- mths_since_recent_revol_delinq: integer (nullable = true)
-- num_accts_ever_120_pd: integer (nullable = true)
-- num_actv_bc_tl: integer (nullable = true)
-- num_actv_rev_tl: integer (nullable = true)
-- num_bc_sats: integer (nullable = true)
-- num_bc_tl: integer (nullable = true)
-- num_il_tl: integer (nullable = true)
-- num_op_rev

In [4]:
X=pd.read_csv("/dbfs/FileStore/tables/loan_top1.csv",usecols=['loan_amnt','funded_amnt','funded_amnt_inv','int_rate','installment','annual_inc','dti','delinq_2yrs','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec','revol_bal','revol_util','total_acc','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_amnt',   'collections_12_mths_ex_med','mths_since_last_major_derog','policy_code','annual_inc_joint','dti_joint','acc_now_delinq','tot_coll_amt','tot_cur_bal','open_acc_6m','open_act_il', 'open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m','acc_open_past_24mths','avg_cur_bal',
'bc_open_to_buy','bc_util','chargeoff_within_12_mths','delinq_amnt','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mort_acc','mths_since_recent_bc','mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl',
'num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m','pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit','revol_bal_joint','sec_app_inq_last_6mths','sec_app_mort_acc','sec_app_open_acc','sec_app_revol_util','sec_app_open_act_il','sec_app_num_rev_accts','sec_app_chargeoff_within_12_mths','sec_app_collections_12_mths_ex_med','deferral_term','hardship_amount','hardship_length','hardship_dpd','orig_projected_additional_accrued_interest','hardship_payoff_balance_amount','hardship_last_payment_amount','settlement_amount','settlement_percentage','settlement_term'
])

In [5]:
#X1=pd.read_csv("/dbfs/FileStore/tables/loan_top.csv",usecols=['loan_amnt','funded_amnt','funded_amnt_inv','term','int_rate','installment','grade','sub_grade','emp_title','emp_length','home_ownership','annual_inc','verification_status','issue_d','loan_status','purpose','title','zip_code','addr_state','dti','delinq_2yrs','earliest_cr_line','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec','revol_bal','revol_util','total_acc','initial_list_status','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','next_pymnt_d','last_credit_pull_d',   'collections_12_mths_ex_med','mths_since_last_major_derog','policy_code','application_type','annual_inc_joint','dti_joint','verification_status_joint','acc_now_delinq','tot_coll_amt','tot_cur_bal','open_acc_6m','open_act_il', 'open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m','acc_open_past_24mths','avg_cur_bal',
#'bc_open_to_buy','bc_util','chargeoff_within_12_mths','delinq_amnt','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mort_acc','mths_since_recent_bc','mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl',
#'num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m','pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit','revol_bal_joint','sec_app_earliest_cr_line','sec_app_inq_last_6mths','sec_app_mort_acc','sec_app_open_acc','sec_app_revol_util','sec_app_open_act_il','sec_app_num_rev_accts','sec_app_chargeoff_within_12_mths','sec_app_collections_12_mths_ex_med','hardship_flag','hardship_type','hardship_reason','hardship_status','deferral_term','hardship_amount','hardship_start_date','hardship_end_date','payment_plan_start_date','hardship_length','hardship_dpd','hardship_loan_status','orig_projected_additional_accrued_interest','hardship_payoff_balance_amount','hardship_last_payment_amount','disbursement_method','debt_settlement_flag','debt_settlement_flag_date','settlement_status','settlement_date','settlement_amount','settlement_percentage','settlement_term'
#])

In [6]:
X.replace(np.nan, 0)


,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,...,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
0,30000,30000,30000,22.35,1151.16,100000.0,30.46,0,0,51.0,84.0,11,1,15603,37.0,19,0,0,30026.442500,30026.44,30000.00,26.44,0.0,0.0,0.0,30082.32,0,0.0,1,150000.0,23.38,0,0,472330,1,3,2,2,2.0,82850,...,3,4,3,5,10,6,8,4,11,0.0,0,0,2,89.5,33.3,1,0,527120,98453,28600,101984,52417.0,0.0,1.0,16.0,25.2,2.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40000,40000,40000,16.14,975.71,45000.0,50.53,0,0,0.0,0.0,18,0,34971,64.5,37,0,0,40856.678960,40856.68,40000.00,856.68,0.0,0.0,0.0,40910.48,0,0.0,1,92000.0,35.66,0,0,271068,2,8,3,4,5.0,126749,...,7,9,7,7,26,9,10,9,18,0.0,0,0,4,100.0,42.9,0,0,344802,161720,45700,167965,47188.0,0.0,1.0,32.0,61.1,16.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20000,20000,20000,7.56,622.68,100000.0,18.92,0,0,48.0,0.0,9,0,25416,29.9,19,0,0,20215.792430,20215.79,20000.00,215.79,0.0,0.0,0.0,20228.39,0,0.0,1,190000.0,11.75,0,0,515779,1,2,0,1,13.0,46153,...,5,5,5,6,5,5,9,5,9,0.0,0,0,1,94.7,20.0,0,0,622183,71569,85100,74833,43287.0,0.0,3.0,10.0,29.7,2.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4500,4500,4500,11.31,147.99,38500.0,4.64,0,0,25.0,0.0,12,0,4472,15.3,25,0,0,4549.217149,4549.22,4500.00,49.22,0.0,0.0,0.0,4553.46,0,25.0,1,0.0,0.00,0,306,29137,2,1,1,1,3.0,24665,...,3,5,5,6,10,11,15,5,12,0.0,0,0,4,91.7,0.0,0,0,53795,29137,15100,24595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8425,8425,8425,27.27,345.18,450000.0,12.37,0,0,0.0,0.0,21,0,36812,65.7,37,0,0,8735.148975,8735.15,8425.00,310.15,0.0,0.0,0.0,8754.30,0,0.0,1,515000.0,16.47,0,0,690320,0,6,1,6,12.0,152382,...,6,8,8,10,16,13,17,8,21,0.0,0,0,2,100.0,50.0,0,0,768304,189194,45800,189054,141905.0,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,20000,20000,20000,17.97,507.55,57000.0,22.18,0,0,0.0,0.0,10,0,33356,45.9,16,0,0,20013.577330,20013.58,20000.00,13.58,0.0,0.0,0.0,20043.53,0,0.0,1,0.0,0.00,0,0,33356,2,0,0,0,78.0,0,...,4,7,6,9,1,10,14,7,10,0.0,0,0,2,100.0,33.3,0,0,72700,33356,64800,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6600,6600,6325,11.31,217.05,45000.0,16.21,0,0,0.0,0.0,6,0,6167,58.2,13,0,0,6622.809000,6346.86,6600.00,22.81,0.0,0.0,0.0,6629.03,0,0.0,1,0.0,0.00,0,0,26836,0,2,0,0,27.0,20669,...,2,2,4,4,5,4,8,2,6,0.0,0,0,0,84.6,50.0,0,0,38607,26836,10600,28007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2500,2500,2475,13.56,84.92,42000.0,15.09,0,0,53.0,0.0,3,0,3319,31.6,6,0,0,2501.285667,2476.27,2500.00,1.29,0.0,0.0,0.0,2504.11,0,0.0,1,0.0,0.00,0,0,18649,1,1,0,1,22.0,15330,...,2,2,2,2,3,2,3,2,3,0.0,0,0,1,83.3,0.0,0,0,32582,18649,10500,22082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [7]:
y=pd.read_csv("/dbfs/FileStore/tables/loan_top1.csv",usecols=['loan_status'])
y.replace(np.nan, 0)
y=y.astype(int)

In [8]:
np.isnan(X.values.any())

Out[9]: False

In [9]:
X[X==np.inf]=np.nan
X.fillna(X.mean(), inplace=True)
#y.fillna(y.mean(), inplace=True)

In [10]:
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['features','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features

features Score
20 total_rec_prncp 7.550476e+06
18 total_pymnt 7.464164e+06
19 total_pymnt_inv 7.462310e+06
82 tot_hi_cred_lim 7.178013e+06
25 last_pymnt_amnt 6.653587e+06
33 tot_cur_bal 6.349094e+06
39 total_bal_il 6.516388e+05
85 total_il_high_credit_limit 6.446203e+05
50 avg_cur_bal 5.359959e+05
51 bc_open_to_buy 4.231476e+05

In [11]:
#from sklearn.ensemble import ExtraTreesClassifier
#import matplotlib.pyplot as plt
#model = ExtraTreesClassifier()
#model.fit(X,y)
#print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
#feat_importances = pd.Series(model.feature_importances_, index=X.columns)
#feat_importances.nlargest(10).plot(kind='barh')
#plt.show()

/databricks/python/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
 "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/local_disk0/tmp/1574894476173-0/PythonShell.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
 import errno
[0.09572415 0.07418522 0.03013702 0.00510243 0.07325266 0.00210706
 0.00342155 0.00241768 0.00149612 0.00109447 0.00038086 0.00199299
 0.00084772 0.00241559 0.00340166 0.00180586 0. 0.
 0.16858627 0.12732621 0.1830679 0.01136426 0.00727238 0.00111478
 0.00222825 0.08240577 0.00208947 0.00140147 0. 0.00150991
 0.00113989 0. 0.00133621 0.00128463 0.00199848 0.00125078
 0.00254374 0.00255116 0.00266731 0.00202101 0.002189 0.00110613
 0.00159127 0.00152091 0.00169086 0.00240197 0.0016299 0.00194053
 0.00248254 0.00165329 0.00225543 0.0023769 0.00419002 0.00054899
 0. 0.00242672 0.00319115 0.00254804 0.00399891 0.00195436
 0.00143345 0.00145683 0.00268115 0.00097816 0.00132132 0.00198296
 0.00290212 0.00111916 0.00122735 0.0017226 0.00275898 0.00178098
 0.0042658 0.00240839 0. 0. 0.00068025 0.00164081
 0.00138856 0.00237909 0.00073201 0. 0.00298548 0.00171904
 0.00175836 0.00177225 0.0010706 0.00030428 0.0011629 0.00088234
 0.00045744 0.00100381 0.00058179 0.00054262 0.00072163 0.
 0. 0. 0. 0. 0. 0.
 0.00123027 0.00170154 0.00060585]